# Linear Regression
这个 notebook 是线性回归的代码实现，提供梯度下降和 Normal Equation 两种方法进行模型的训练。在模型优化中，提供 batch gradient descent、mini-batch gradient descent 和 stochastic gradient descent 三种方法

## 假设函数 $h$ 的实现
输入 $X$ 是一个 $m$ 行 $n$ 列的矩阵，每一行是一个样本的 $n$ 个属性的参数。$\boldsymbol \theta$ 是我们待求的参数，是一个 $n$ 维向量，假设函数的计算公式如下：
$$
h(X) =  X  \boldsymbol \theta
$$

In [1]:
import numpy as np
import math

In [2]:
# 函数功能：计算假设函数的结果，公式如上
#     输入：X (m, n) 是一个 m 行 n 列的矩阵，m 是样本个数，n 是 theta 的维度
#           parameters 是一个 (n, 1) 的 n 维向量，其就是公式中的 theta
#     输出：y (m, 1) 是一个 m 维的列向量，每一行是一个样本的输出
def hypothese(X, parameters):
    y = np.dot(X, parameters)
    return y

In [3]:
#函数功能：将训练集数据按照 miniBatchSize 划分成多个小批数据，默认为 64
#    输入：X 输入数据集，(m, n)
#          y 输出数据集，(m, 1)
#          miniBatchSize，划分后每个小批数据的个数，默认为 64
#          seed 随机数种子，默认为 0
#    输出：miniBatchs 划分好的数据集，放在一个列表中
def randomMiniBatchs(X, y, miniBatchSize = 64, seed = 0):
    m = X.shape[0]             #取得样本个数 m
    np.random.seed(seed)       #设置随机数种子
    miniBatchs = []            #存放分类好的小批数据集
    miniBatch_X = []           #临时存放一个重排序后的小批的输入数据集
    miniBatch_y = []           #临时存放一个重排序后的小批的标签数据集
    miniBatch = ()             #临时存放一个批
    
    permutation = list(np.random.permutation(m))     #将索引随机重排列
    
    numCompleteBatchs = math.floor(m / miniBatchSize)   #存放完整的批的个数
    for i in range(0, numCompleteBatchs):
        miniBatch_X = X[permutation[i * miniBatchSize : (i + 1) * miniBatchSize] ,:]
        miniBatch_y = y[permutation[i * miniBatchSize : (i + 1) * miniBatchSize] ,:]
        miniBatch = (miniBatch_X, miniBatch_y)     #将 miniBatch_X 和 miniBatch_y 打包
        miniBatchs.append(miniBatch)               #将这一批数据放入 miniBatchs 中
    
    if(m % miniBatchSize != 0):
        miniBatch_X = X[permutation[numCompleteBatchs * miniBatchSize : ] ,:]   #将剩下的数据组成一个批
        miniBatch_y = y[permutation[numCompleteBatchs * miniBatchSize : ] ,:]   
        miniBatch = (miniBatch_X, miniBatch_y)
        miniBatchs.append(miniBatch)
    
    return miniBatchs  

In [7]:
#函数功能：在数据前增加一列或一行全为1
#    输入：X_train 原始数据 (m, n)
#          ax = 1表示在最左边加1列， = 0 表示在最上面加一行
#    输出：X 加完后的数组  (m + 1, n) 或 (m, n + 1)
def expendOneCol(X_train, ax = 1):
    X = np.insert(X_train, 0, values = np.ones((1,X_train.shape[(ax+1)%2])), axis = ax)
    return X;

In [8]:
#函数功能：初始化参数
#    输入：m 维度
#    输出：parameters 初始化为 0 的数组 (m, 1)
def initialParameters(m):
    parameters = np.zeros((m, 1))
    return parameters

In [9]:
#函数功能：计算梯度
#    输入：X_train 训练数据
#          y 标签数据
#          h 假设函数结果
#    输出：grad 梯度
def calculateGrad(X_train, y, h):
    grad = np.dot(X_train.T, h - y)
    return grad

In [10]:
#函数功能：计算损失值
#    输入：y 标签
#          h 假设函数结果
#    输出：J 损失值
def calLossFunction(y, h):
    sub = (h - y)
    J = np.dot(sub.T, sub) / 2
    return J

In [31]:
#函数功能：线性回归训练，可调 batchSize 进行 SGD、Mini batch GD 或 BGD
#    输入：X_train 是一个 m 行 n 列矩阵，是训练数据
#          y_train 是 X_train 对应的标签，维度是 (m, 1)
#          parameters 是训练参数，维度是 (m, 1)
#          learningRate 是学习率
#          miniBatchSize 是每次更新的批样本数、当 miniBatchSize = 1 时进行的是随机梯度下降、当 miniBatchSize = 其他值进行的是 mini-batch GD
#          iteration 是迭代循环更新的次数
#    输出：训练好的参数 parameters
def linearRegressionTrain(X_train, y_train, parameters, learningRate, iteration, miniBatchSize = 64, seed = 0):
    miniBatchs = []    #用于存放随机重组的小批数据
    miniBatch_X = []   #用于临时存放一批数据的输入
    miniBatch_y = []   #用来临时存放一批数据的标签
    
    miniBatchs = randomMiniBatchs(X_train, y_train, miniBatchSize, seed)   #将数据分批
    
    for itera in range(0, iteration):
        for batchNum in range(0, len(miniBatchs)):
            (miniBatch_X, miniBatch_y) = miniBatchs[batchNum]   #取出第 batchNum 个批的数据
            h = hypothese(miniBatch_X, parameters)              #计算这一批数据的假设函数的结果
            grad = np.dot(miniBatch_X.T, h - miniBatch_y)       #计算梯度
            parameters = parameters - learningRate * grad       #更新参数
            
    
    return parameters

In [32]:
def model(X_train, y):
    X_train = expendOneCol(X_train)
    (m, n) = X_train.shape
    
    parameters = initialParameters(n)
    
    parameters = linearRegressionTrain(X_train, y, parameters, 0.01, 10)
    
    return parameters

In [33]:
import pandas as pd   
import numpy as np

data_X = pd.read_csv('./r_train_X.csv', encoding = 'big5')  #r_train_X.csv文件要与python代码文件(.py或.ipynb)在同一个目录下才行
data_y = pd.read_csv('./r_train_y.csv', encoding = 'big5')  #r_train_y.csv文件要与python代码文件(.py或.ipynb)在同一个目录下才行
X = data_X.to_numpy()  #将数据转换成numpy数据
y = data_y.to_numpy()  #将数据转换成numpy数据

In [34]:
p = model(X, y)

In [35]:
p

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
      